# 🌞 Example Weather App
> Using `prodb`


* Starting with a `.csv` of locations, we wish to call the [MetaWeather](https://www.metaweather.com/api/) API to pull in the weather forecast.  
* The app needs the ability to input new locations by the user

 🌍 Core functions will be converted into the streamlit app `weather_app.py`

In [ ]:
# default_exp examples

In [ ]:
#hide
from nbdev.showdoc import *
import pandas as pd
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Requests to `metaweather` open weather API

In [ ]:
import requests
import json
import arrow

In [ ]:
#export
def get_json_from_query(location):
    """Search for a city and return metadata from API"""
    url = f"https://www.metaweather.com/api/location/search/?query={location}"
    r = requests.get(url).json()
    return r[0]


In [ ]:
l = get_json_from_query('London')
l

{'title': 'London',
 'location_type': 'City',
 'woeid': 44418,
 'latt_long': '51.506321,-0.12714'}

Check `woeid`(Where On Earth ID) is correct for London

In [ ]:
assert l['woeid'] == 44418

Get weather data for the London `woeid`

In [ ]:
utc = arrow.utcnow().format('YYYY/MM/DD')
utc

'2021/11/05'

In [ ]:
#export
def get_current_weather(location):
    """
    inputs: location str "London"
            to find woeid i.e. 44418
    """
    res = get_json_from_query(location)
    woeid = res['woeid']
    url = f"https://www.metaweather.com/api/location/{woeid}/"
    res = requests.get(url).json()
    return res

Today's forecast 🌤️

In [ ]:
%%time
res = get_current_weather('London')
res['consolidated_weather'][0]

Wall time: 2.38 s


{'id': 6607174022201344,
 'weather_state_name': 'Heavy Cloud',
 'weather_state_abbr': 'hc',
 'wind_direction_compass': 'WNW',
 'created': '2021-11-05T18:59:01.554259Z',
 'applicable_date': '2021-11-05',
 'min_temp': 2.915,
 'max_temp': 10.14,
 'the_temp': 9.030000000000001,
 'wind_speed': 4.357755747228187,
 'wind_direction': 281.959650545896,
 'air_pressure': 1027.0,
 'humidity': 67,
 'visibility': 7.677973136880617,
 'predictability': 71}

There are a number of metrological properties available to us from the API. 

The ones we are most interested in are:
* `the_temp` 🌡️ current temperature
* `weather_state_name` ⛅ current sky condition
* `max_temp` 🥵 daily max temp
* `min_temp` 🥶 daily min temp


In [ ]:
res = get_current_weather('London')
df =  pd.DataFrame.from_records(res['consolidated_weather'][:1])
df['readable_time'] = df.created.apply(lambda x: arrow.get(x).humanize())
df

,id,weather_state_name,weather_state_abbr,wind_direction_compass,created,applicable_date,min_temp,max_temp,the_temp,wind_speed,wind_direction,air_pressure,humidity,visibility,predictability,readable_time
0,6607174022201344,Heavy Cloud,hc,WNW,2021-11-05T18:59:01.554259Z,2021-11-05,2.915,10.14,9.03,4.357756,281.959651,1027.0,67,7.677973,71,2 hours ago


In [ ]:
#export
def df_from_loc(location, 
                days_ahead=1,
                keep_cols='location the_temp readable_time created applicable_date local_time latt_long weather_state_name min_temp max_temp'.split(' ')):
    res = get_current_weather(location)
    df =  pd.DataFrame.from_records(res['consolidated_weather'][:days_ahead])
    df['location'] = location
    df['local_time'] = pd.to_datetime(res['time']).strftime('%H:%M (%Y-%m-%d)')
    df['latt_long'] = res['latt_long']
    df['readable_time'] = df.created.apply(lambda x: arrow.get(x).humanize())
    return df[keep_cols]


In [ ]:
df = df_from_loc('London', days_ahead=2)
df

,location,the_temp,readable_time,created,applicable_date,local_time,latt_long,weather_state_name,min_temp,max_temp
0,London,9.030,2 hours ago,2021-11-05T18:59:01.554259Z,2021-11-05,21:37 (2021-11-05),"51.506321,-0.12714",Heavy Cloud,2.915,10.140
1,London,12.045,2 hours ago,2021-11-05T18:59:01.554659Z,2021-11-06,21:37 (2021-11-05),"51.506321,-0.12714",Heavy Cloud,6.490,12.985


Example output showing location and current weather situation

In [ ]:
df = df_from_loc('Auckland')
df

,location,the_temp,readable_time,created,applicable_date,local_time,latt_long,weather_state_name,min_temp,max_temp
0,Auckland,19.67,2 hours ago,2021-11-05T19:37:04.232519Z,2021-11-06,10:37 (2021-11-06),"-36.884109,174.770416",Light Rain,13.155,20.35


# 🦄 Pro db 

In [ ]:
import sys
sys.path.append('../')

from prodb.core import generate_db, insert_row

### Initialise empty db

In [ ]:
dbpath = 'db.csv'
cols = 'location temp high low weather_state'.split()

generate_db(cols=cols, dbpath=dbpath)
df = pd.read_csv('db.csv')
df.head()

✓💾 db.csv (38 kb)


,location,temp,high,low,weather_state


## Add row

In [ ]:
dx = df_from_loc('Auckland')
dx

,location,the_temp,readable_time,created,applicable_date,local_time,latt_long,weather_state_name,min_temp,max_temp
0,Auckland,19.67,2 hours ago,2021-11-05T19:37:04.232519Z,2021-11-06,10:42 (2021-11-06),"-36.884109,174.770416",Light Rain,13.155,20.35


In [ ]:
dx = df_from_loc('Christchurch')

data = {'location': dx.location.item(),
        'time_utc': arrow.utcnow().format('YYYY-MM-DD HH:mm:ss'),
        'temp': dx.the_temp.item(),
        'high': dx.max_temp.item(),
        'low': dx.min_temp.item(),
        'weather_state': dx.weather_state_name.item(),
        'local_time': dx.local_time.item()}

df = insert_row(df, data)
display(df)

,location,temp,high,low,weather_state,local_time,time_utc
0,Christchurch,15.95,15.385,9.54,Heavy Cloud,10:42 (2021-11-06),2021-11-05 21:42:27


In [ ]:
#export

def visit_city(df, cities):
    if isinstance(cities, str): cities = [cities]
    for city in cities:
        dx = df_from_loc(city).round(1)

        data = {'location': dx.location.item(), 
                'time_utc': arrow.utcnow().format('YYYY-MM-DD HH:mm:ss'),
                'temp': dx.the_temp.item(),
                'high': dx.max_temp.item(),
                'low': dx.min_temp.item(),
                'weather_state': dx.weather_state_name.item(),
                'local_time': dx.local_time.item()}

        df = insert_row(df, data)
    return df
        

In [ ]:
df = visit_city(df, ['Wellington', 'Lagos', 'Zagreb'])
display(df)

,location,temp,high,low,weather_state,local_time,time_utc
0,Christchurch,15.95,15.385,9.54,Heavy Cloud,10:42 (2021-11-06),2021-11-05 21:42:27
1,Wellington,16.90,16.200,12.50,Showers,10:42 (2021-11-06),2021-11-05 21:42:33
2,Lagos,29.80,30.400,25.30,Heavy Rain,22:42 (2021-11-05),2021-11-05 21:42:36
3,Zagreb,10.10,11.100,7.80,Showers,22:42 (2021-11-05),2021-11-05 21:42:39


In [ ]:
df = visit_city(df, 'Wellington')
display(df)

,location,temp,high,low,weather_state,local_time,time_utc
0,Christchurch,15.95,15.385,9.54,Heavy Cloud,10:42 (2021-11-06),2021-11-05 21:42:27
1,Wellington,16.90,16.200,12.50,Showers,10:42 (2021-11-06),2021-11-05 21:42:33
2,Lagos,29.80,30.400,25.30,Heavy Rain,22:42 (2021-11-05),2021-11-05 21:42:36
3,Zagreb,10.10,11.100,7.80,Showers,22:42 (2021-11-05),2021-11-05 21:42:39
4,Wellington,16.90,16.200,12.50,Showers,10:42 (2021-11-06),2021-11-05 21:42:43


In [ ]:
df = visit_city(df, ['Singapore', 'Alexandria', 'Bangkok'])
display(df)

,location,temp,high,low,weather_state,local_time,time_utc
0,Christchurch,15.95,15.385,9.54,Heavy Cloud,10:42 (2021-11-06),2021-11-05 21:42:27
1,Wellington,16.90,16.200,12.50,Showers,10:42 (2021-11-06),2021-11-05 21:42:33
2,Lagos,29.80,30.400,25.30,Heavy Rain,22:42 (2021-11-05),2021-11-05 21:42:36
3,Zagreb,10.10,11.100,7.80,Showers,22:42 (2021-11-05),2021-11-05 21:42:39
4,Wellington,16.90,16.200,12.50,Showers,10:42 (2021-11-06),2021-11-05 21:42:43
5,Singapore,28.90,29.500,25.30,Heavy Rain,05:42 (2021-11-06),2021-11-05 21:42:44
6,Alexandria,24.90,25.000,21.30,Clear,23:42 (2021-11-05),2021-11-05 21:42:46
7,Bangkok,33.40,33.400,25.40,Light Rain,04:42 (2021-11-06),2021-11-05 21:42:47


In [ ]:
from prodb.core import 